In [12]:
import tempfile
import pandas as pd

from gtfparse import read_gtf
from Bio import SeqIO
from datapaths import *
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', None)

In [13]:
flipon_to_data = pd.read_table(F_FLIPON_TO_DATA)
flipon_to_data

,Flipon,Coordinates,Strand,Gene Feature,Gene Name,Gene Strand,Gene Type,Gene ID,Transcript ID,c. M miR (+),c. M miR (-),c. MJ miR (+),c. MJ miR (-),c. J miR (+),c. J miR (-),all M miR (+),all M miR (-),all MJ miR (+),all MJ miR (-),all J miR (+),all J miR (-),cCRE (+-200bp),LINE/LTR (+-200bp),Feature Group
0,g4,chr1:3014794-3014871,-,Distal Intergenic,4933401J01Rik,+,TEC,ENSMUSG00000102693.1,ENSMUST00000193812.1,miR-328 (1),NaN,miR-486 (1),NaN,NaN,NaN,"miR-6994 (1), miR-129b (1)","miR-92a-2 (1), miR-698/miR-7078 (1), miR-3572 ...",miR-486a/miR-486b (1),"miR-3076 (1), miR-6418 (1), miR-6987 (1)",NaN,miR-7649 (1),NaN,"LINE@L1Md_F2(-), LINE@L1VL1(+)",LINE/LTR
1,g4,chr1:3099888-3099963,+,Promoter (2-3kb),Gm26206,+,snRNA,ENSMUSG00000064842.1,ENSMUST00000082908.1,NaN,miR-328 (1),NaN,NaN,NaN,NaN,"miR-365-1/miR-365-2 (1), miR-698/miR-7078 (1),...","miR-221 (1), miR-129b (1)","miR-6418 (1), miR-7067 (1)",NaN,NaN,NaN,NaN,"LINE@L1Md_F2(+), LTR@MTC(+)",LINE/LTR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77706,z-dna,chrY:4203100-4203146,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,miR-493 (1),NaN,NaN,NaN,"miR-466d/miR-466i (4), miR-466l (5)",NaN,miR-493 (1),NaN,NaN,NaN,NaN,LINE@L1_Mur1(-),LINE/LTR
77707,z-dna,chrY:4207093-4207113,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,NaN,NaN,NaN,NaN,"miR-466a/miR-466e/miR-466p (1), miR-466d/miR-4...",NaN,NaN,NaN,NaN,NaN,NaN,"LINE@L1_Mus3(+), LINE@L1Md_A(+), LTR@ORR1B1(+)",LINE/LTR


In [14]:
# gene_feature_order = ["Promoter (<=1kb)", "Promoter (1-2kb)", "Promoter (2-3kb)", "5' UTR", "Exon", "Intron", "3' UTR", "Downstream (<=300bp)", "Distal Intergenic"]
# pivot_column_order = [(gene_feature, mir_type) for gene_feature in gene_feature_order for mir_type in mir_order]

In [47]:
from itertools import combinations

mir_groups = ['M', 'MJ', 'J']
mir_group_combinations = list()
for mir_type in ['c.', 'all', 'c. + all']:
    if '+' not in mir_type:
        for r in range(1,4):
            cur_combinations = combinations(mir_groups, r=r)
            for groups in cur_combinations:
                temp = []
                for group in groups:
                    temp += [f'{mir_type} {group} miR (+)', f'{mir_type} {group} miR (-)']
                mir_group_combinations.append((mir_type, '+'.join(groups),temp))
    else:
        first = mir_type.split(' + ')[0]
        second = mir_type.split(' + ')[1]
        for r in range(1,4):
            cur_combinations = combinations(mir_groups, r=r)
            for groups in cur_combinations:
                temp = []
                for group in groups:
                    temp += [f'{first} {group} miR (+)', f'{first} {group} miR (-)', f'{second} {group} miR (+)', f'{second} {group} miR (-)']
                mir_group_combinations.append((mir_type, '+'.join(groups),temp))

mir_group_combinations

[('c.', 'M', ['c. M miR (+)', 'c. M miR (-)']),
 ('c.', 'MJ', ['c. MJ miR (+)', 'c. MJ miR (-)']),
 ('c.', 'J', ['c. J miR (+)', 'c. J miR (-)']),
 ('c.',
  'M+MJ',
  ['c. M miR (+)', 'c. M miR (-)', 'c. MJ miR (+)', 'c. MJ miR (-)']),
 ('c.',
  'M+J',
  ['c. M miR (+)', 'c. M miR (-)', 'c. J miR (+)', 'c. J miR (-)']),
 ('c.',
  'MJ+J',
  ['c. MJ miR (+)', 'c. MJ miR (-)', 'c. J miR (+)', 'c. J miR (-)']),
 ('c.',
  'M+MJ+J',
  ['c. M miR (+)',
   'c. M miR (-)',
   'c. MJ miR (+)',
   'c. MJ miR (-)',
   'c. J miR (+)',
   'c. J miR (-)']),
 ('all', 'M', ['all M miR (+)', 'all M miR (-)']),
 ('all', 'MJ', ['all MJ miR (+)', 'all MJ miR (-)']),
 ('all', 'J', ['all J miR (+)', 'all J miR (-)']),
 ('all',
  'M+MJ',
  ['all M miR (+)', 'all M miR (-)', 'all MJ miR (+)', 'all MJ miR (-)']),
 ('all',
  'M+J',
  ['all M miR (+)', 'all M miR (-)', 'all J miR (+)', 'all J miR (-)']),
 ('all',
  'MJ+J',
  ['all MJ miR (+)', 'all MJ miR (-)', 'all J miR (+)', 'all J miR (-)']),
 ('all',
  'M+MJ

In [48]:
summary_conserved = flipon_to_data[['Flipon', 'Gene Feature']].copy()

summary_conserved['Any'] = True
for mir_type, groups, mir_group_combination in mir_group_combinations:
    if mir_type == 'c.':
        summary_conserved[groups] = (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)

summary_conserved = summary_conserved.groupby(['Flipon', 'Gene Feature']).sum().reset_index()
summary_conserved = summary_conserved.melt(id_vars=['Flipon', 'Gene Feature'], value_vars=summary_conserved.columns[2:], var_name='mir_group', value_name='total')
summary_conserved['mir_type'] = 'conserved'
display(summary_conserved)

,Flipon,Gene Feature,mir_group,total,mir_type
0,g4,3' UTR,Any,264,conserved
1,g4,5' UTR,Any,13,conserved
...,...,...,...,...,...
286,z-dna,Promoter (2-3kb),M+MJ+J,433,conserved
287,z-dna,Promoter (<=1kb),M+MJ+J,3687,conserved


In [57]:
summary_all = flipon_to_data[['Flipon', 'Gene Feature']].copy()

summary_all['Any'] = True
for mir_type, groups, mir_group_combination in mir_group_combinations:
    if mir_type == 'all':
        summary_all[groups] = (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)

summary_all = summary_all.groupby(['Flipon', 'Gene Feature']).sum().reset_index()
summary_all = summary_all.melt(id_vars=['Flipon', 'Gene Feature'], value_vars=summary_all.columns[2:], var_name='mir_group', value_name='total')
summary_all['mir_type'] = 'all'
display(summary_all)

,Flipon,Gene Feature,mir_group,total,mir_type
0,g4,3' UTR,Any,264,all
1,g4,5' UTR,Any,13,all
...,...,...,...,...,...
286,z-dna,Promoter (2-3kb),M+MJ+J,791,all
287,z-dna,Promoter (<=1kb),M+MJ+J,8080,all


In [58]:
summary_both = flipon_to_data[['Flipon', 'Gene Feature']].copy()

summary_both['Any'] = True
for mir_type, groups, mir_group_combination in mir_group_combinations:
    if mir_type == 'c. + all':
        summary_both[groups] = (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)

summary_both = summary_both.groupby(['Flipon', 'Gene Feature']).sum().reset_index()
summary_both = summary_both.melt(id_vars=['Flipon', 'Gene Feature'], value_vars=summary_both.columns[2:], var_name='mir_group', value_name='total')
summary_both['mir_type'] = 'conserved + all'
display(summary_both)

,Flipon,Gene Feature,mir_group,total,mir_type
0,g4,3' UTR,Any,264,conserved + all
1,g4,5' UTR,Any,13,conserved + all
...,...,...,...,...,...
286,z-dna,Promoter (2-3kb),M+MJ+J,804,conserved + all
287,z-dna,Promoter (<=1kb),M+MJ+J,8285,conserved + all


In [68]:
gene_feature_order = ["Promoter (<=1kb)", "Promoter (1-2kb)", "Promoter (2-3kb)", "5' UTR", "Exon", "Intron", "3' UTR", "Downstream (<=300bp)", "Distal Intergenic"]
mir_type_order = ['conserved', 'all', 'conserved + all']
mir_group_order = ['Any', 'M', 'MJ', 'J', 'M+MJ', 'M+J', 'MJ+J', 'M+MJ+J']
pivot_column_order = [(gene_feature, mir_type, mir_group) for gene_feature in gene_feature_order for mir_type in mir_type_order for mir_group in mir_group_order]

In [69]:
summary_total = pd.concat([summary_conserved, summary_all, summary_both], ignore_index=True)
summary_total = summary_total.pivot_table(index='Flipon', columns=['Gene Feature', 'mir_type', 'mir_group'], values=['total']).droplevel(0, axis=1)

summary_total = summary_total[pivot_column_order]

summary_total.to_csv(D_TABLES / 'flipons_summary.tsv', sep='\t')
summary_total

Gene Feature Promoter (<=1kb)                                                  \
mir_type            conserved                                             all   
mir_group                 Any     M    MJ    J  M+MJ   M+J  MJ+J M+MJ+J   Any   
Flipon                                                                          
g4                       6565  2674  1949  339  3758  2877  2181   3908  6565   
h-dna                    1767   269   400   85   586   346   464    643  1767   
sidd                      860   790   214   49   805   795   253    810   860   
z-dna                    9365  1468  2311  497  3367  1893  2680   3687  9365   

Gene Feature                                                                   \
mir_type                                                conserved + all         
mir_group        M    MJ     J  M+MJ   M+J  MJ+J M+MJ+J             Any     M   
Flipon                                                                          
g4            6348  5813  1359  6512  6387  5919   6517            6565  6369   
h-dna         1431  1504   624  1618  1508  1567   1646            1767  1431   
sidd           860   657   253   860   860   710    860             860   860   
z-dna         4628  6464  2540  7734  5915  7093   8080            9365  4826   

Gene Feature                                      Promoter (1-2kb)            \
mir_type                                                 conserved             
mir_group       MJ     J  M+MJ   M+J  MJ+J M+MJ+J              Any    M   MJ   
Flipon                                                                         
g4            5877  1488  6520  6408  5978   6526             1008  395  309   
h-dna         1514   650  1620  1514  1576   1649              679  102  147   
sidd           685   273   860   860   739    860              559  529  166   
z-dna         6793  2725  7962  6151  7391   8285             1447  282  429   

Gene Feature                                                               \
mir_type                                  all                               
mir_group       J M+MJ  M+J MJ+J M+MJ+J   Any    M    MJ    J  M+MJ   M+J   
Flipon                                                                      
g4             56  569  430  351    595  1008  983   905  224  1003   989   
h-dna          30  215  127  170    234   679  558   562  260   622   589   
sidd           32  538  530  184    538   559  558   452  207   559   559   
z-dna         107  606  363  499    660  1447  869  1018  364  1250  1006   

Gene Feature                                                                 \
mir_type                  conserved + all                                     
mir_group     MJ+J M+MJ+J             Any    M    MJ    J  M+MJ   M+J  MJ+J   
Flipon                                                                        
g4             919   1003            1008  984   922  252  1003   990   936   
h-dna          588    632             679  558   570  267   624   590   597   
sidd           481    559             559  558   474  214   559   559   498   
z-dna         1107   1295            1447  909  1087  427  1290  1057  1171   

Gene Feature        Promoter (2-3kb)                                           \
mir_type                   conserved                                      all   
mir_group    M+MJ+J              Any    M   MJ   J M+MJ  M+J MJ+J M+MJ+J  Any   
Flipon                                                                          
g4             1003              673  264  223  45  374  288  255    392  673   
h-dna           634              566  107  126  37  200  136  151    218  566   
sidd            559              458  440  126  27  447  441  146    447  458   
z-dna          1326              863  208  281  70  395  259  333    433  863   

Gene Feature                                                                 \
mir_type                                          conserved + all             
mir_

In [60]:
# summary = flipon_to_data[['Flipon', 'Coordinates', 'Gene Feature']].copy()

# summary['All'] = True
# for mir_type in mir_order[1:]:
#     summary[mir_type] = flipon_to_data[f"{mir_type} (+)"].notna() | flipon_to_data[f"{mir_type} (-)"].notna()

# summary = summary.groupby(['Flipon', 'Gene Feature'])[mir_order].sum().reset_index()
# summary = summary.melt(id_vars=['Flipon', 'Gene Feature'], value_vars=mir_order, var_name='type', value_name='total')
# summary = summary.pivot_table(index='Flipon', columns=['Gene Feature', 'type'], values=['total']).droplevel(0, axis=1)

# summary = summary[pivot_column_order]

# summary

In [86]:
custom_pivot_column_order = [(mir_type, mir_group) for mir_type in mir_type_order for mir_group in mir_group_order]
custom_pivot_column_order

[('conserved', 'Any'),
 ('conserved', 'M'),
 ('conserved', 'MJ'),
 ('conserved', 'J'),
 ('conserved', 'M+MJ'),
 ('conserved', 'M+J'),
 ('conserved', 'MJ+J'),
 ('conserved', 'M+MJ+J'),
 ('all', 'Any'),
 ('all', 'M'),
 ('all', 'MJ'),
 ('all', 'J'),
 ('all', 'M+MJ'),
 ('all', 'M+J'),
 ('all', 'MJ+J'),
 ('all', 'M+MJ+J'),
 ('conserved + all', 'Any'),
 ('conserved + all', 'M'),
 ('conserved + all', 'MJ'),
 ('conserved + all', 'J'),
 ('conserved + all', 'M+MJ'),
 ('conserved + all', 'M+J'),
 ('conserved + all', 'MJ+J'),
 ('conserved + all', 'M+MJ+J')]

In [87]:
t = pd.concat([summary_conserved, summary_all, summary_both], ignore_index=True)

# t = t[t['mir_group'] == 'Any']

t = t.groupby(['Flipon', 'mir_type', 'mir_group'])['total'].sum().to_frame().reset_index()

t = t.pivot_table(index=['Flipon'], columns=['mir_type', 'mir_group'], values=['total']).droplevel(0, axis=1)

t = t[custom_pivot_column_order]

t

mir_type  conserved                                                  all  \
mir_group       Any      M    MJ     J   M+MJ    M+J  MJ+J M+MJ+J    Any   
Flipon                                                                     
g4            20253   7999  6807  1412  11748   8854  7841  12388  20253   
h-dna         17100   2821  3432   854   5356   3507  4094   5876  17100   
sidd          15296  14725  3945   905  14867  14764  4572  14894  15296   
z-dna         25059   6145  8005  1761  11495   7379  9162  12348  25059   

mir_type                                                  conserved + all  \
mir_group      M     MJ     J   M+MJ    M+J   MJ+J M+MJ+J             Any   
Flipon                                                                      
g4         19790  18498  4332  20157  19870  18729  20168           20253   
h-dna      14061  14347  6516  15591  14634  14886  15841           17100   
sidd       15294  12471  5022  15296  15296  13199  15296           15296   
z-dna      16573  17957  6375  21989  18712  19448  22628           25059   

mir_type                                                   
mir_group      M     MJ     J   M+MJ    M+J   MJ+J M+MJ+J  
Flipon                                                     
g4         19835  18697  5145  20176  19909  18916  20184  
h-dna      14064  14465  6753  15626  14705  15016  15906  
sidd       15295  12910  5386  15296  15296  13572  15296  
z-dna      16987  19261  7328  22564  19247  20603  23161

In [61]:
print('Total coverage')

result = pd.DataFrame()

all = flipon_to_data['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(all.values, index=all.index, name='All')], axis=1)

for mir_type in mir_order[1:]:
    s = flipon_to_data[flipon_to_data[f'{mir_type} (+)'].notna() | flipon_to_data[f'{mir_type} (-)'].notna()]['Flipon'].value_counts().sort_index()
    result = pd.concat([result, pd.Series(s.values, index=s.index, name=mir_type)], axis=1)

display(result)